In [ ]:
#MNIST1D
#Dependencies
from sklearn.linear_model import LinearRegression
from math import factorial

#Compute n choose k, used for weighting factors
def comb(n, k):
    return factorial(n) / factorial(k) / factorial(n - k)

#My SHAP function for MNIST1D
def my_shap(inputs, labels, model, index):
  #Identify input being evaluated
  input = inputs[index]
  label = labels[index]
  #Create explanation map array
  exmap = np.zeros(input.size)
  #Specify Number of features
  M = 40
  #Create arrays to feed into linear regression
  # x is the coalition vector
  x = []
  # y is the prediction based on the coalition vector
  y = []

  
  # create all coalition vectors to be sampled
  for a in range(input.size):
    # initialize coalition vector
    feat = np.zeros(input.size)
    # set one (a-th) feature on
    feat[a] = 1
    # compile coalition vectors including only the ath feature and excluding only the ath feature
    x.append(feat)
    x.append(1 - feat)
  
  # get predictions for all coalition vectors from the model
  for e in range(np.shape(x)[0]):
    # create temporary input vector
    new_input = np.zeros(input.size)
    # fill input vector
    for f in range(input.size):
      # if the feature is included, sample feature from the input being evaluated
      # if the feature is not included (else), sample a random feature from the dataset
      if x[e][f] == 1:
        new_input[f] = input[f]
      else:
        # set edge case to 0, average for rest
        if f == 0 | f == input.size -1:
          new_input[f] = 0
        else:
          new_input[f] = (input[f-1]+input[f+1])/2
    # generate prediction for the new input vector
    pred = model.predict(np.expand_dims(np.expand_dims(new_input, axis=0), axis=-1))
    # compile predictions into array
    y.append(pred)

  # fix dimensions of array to work with linear regression
  y = np.asarray(y)
  y = y[:,:,label]
  
  # create weight vector
  pie = np.zeros(np.shape(x)[0])
  # calculate weights
  for p in range(pie.size):
    z_prime = sum(x[p])
    pie[p] = (M-1) / (z_prime * (M - z_prime) * comb(M, z_prime))

  # find linear regression between predictions and coalition vectors, with weightings
  shap = LinearRegression().fit(x,y,pie)

  # create explanation map
  exmap = shap.coef_.flatten()
  exmap -= np.min(exmap)
  exmap /= np.max(exmap)
  
  return exmap

In [ ]:
#HMT
#import libraries to handle linear regression and computation of combinations
from sklearn.linear_model import LinearRegression
from math import factorial

#Compute n choose k, used for weighting factors
def comb(n, k):
    return factorial(n) / factorial(k) / factorial(n - k)

#My SHAP function for HMT
def my_shap(inputs, labels, model, index):
  #Identify input being evaluated
  input = inputs[index]
  label = labels[index]
  #Specify Number of features, create super cells
  M = 49
  #Create arrays to feed into linear regression
  # x is the coalition vector
  x = []
  # y is the prediction based on the coalition vector
  y = []


  # create all coalition vectors to be sampled
  for b in range(M):
    # initialize coalition vector
    feat = np.zeros(M)
    # set one (b-th) feature on
    feat[b] = 1
    # compile coalition vectors including only the bth feature and excluding only the bth feature
    x.append(feat)
    x.append(1 - feat)


  # get predictions for all coalition vectors from the model
  for e in range(np.shape(x)[0]):
    # create temporary input vector
    new_input = np.zeros((224,224,3))
    # if the feature is included, sample feature from the input being evaluated
    # if the feature is not included (else), zero out the rest of the image
    # fill input vector
    for f in range(224):
      for g in range(224):
        val = int(7*np.floor(f/32)+ np.floor(g/32))
        if x[e][val] == 1:
          new_input[f][g][:] += input[f][g][:]
    # generate prediction for the new input vector
    pred = model.predict(np.expand_dims(new_input, axis=0))
    # compile predictions into array
    y.append(pred)

  # fix dimensions of array to work with linear regression
  y = np.asarray(y)
  y = y[:,:,label.argmax()]
  
  # create weight vector
  pie = np.zeros(np.shape(x)[0])

  # calculate weights
  for p in range(pie.size):
    #scaling factor to make factorial computable
    z_prime = sum(x[p])
    pie[p] = (M-1) / (z_prime * (M - z_prime) * comb(M, z_prime))

  # find linear regression between predictions and coalition vectors, with weightings
  shap = LinearRegression().fit(x,y,pie)

  ## create explanation map, create 224 x 224 representation
  exmap = shap.coef_
  exmap -= np.min(exmap)
  exmap /= np.max(exmap)
  exmap = exmap.reshape(7,7)
  exmap = np.kron(exmap,np.ones((32,32)))
  
  return exmap